Import patra trabajar con GCP

In [ ]:
pip install apache-beam[gcp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.3 MB 5.3 MB/s 
     |████████████████████████████████| 46 kB 4.8 MB/s 
     |████████████████████████████████| 151 kB 64.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 508 kB 55.2 MB/s 
     |████████████████████████████████| 256 kB 59.4 MB/s 
     |████████████████████████████████| 2.3 MB 38.6 MB/s 
     |████████████████████████████████| 233 kB 49.5 MB/s 
     |████████████████████████████████| 180 kB 61.4 MB/s 
     |████████████████████████████████| 255 kB 46.3 MB/s 
     |████████████████████████████████| 180 kB 55.7 MB/s 
     |████████████████████████████████| 267 kB 54.6 MB/s 
     |████████████████████████████████| 265 kB 59.7 MB/s 
     |████████████████████████████████| 183 kB 46.3 MB/s 
     |████████████████████████████████| 435 kB 51.7 MB/s 
     |█████████████████████████████

In [ ]:
from google.oauth2 import service_account #For credentials
from google.cloud import bigquery #For bigquerycredentials
import apache_beam as beam #For apacheBeam
from apache_beam.options.pipeline_options import PipelineOptions #For beam.io.ReadFromBigQuery
#import math #Para calcular el coseno

##Mount access to Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

#Set credentials as global
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="drive/MyDrive/Ideas/movilidad/vacio-276411-service_account_for_colab.json"

##Get credentials
key_path = 'drive/MyDrive/Ideas/movilidad/vacio-276411-service_account_for_colab.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

#Options for ReadFromBigQuery
options = PipelineOptions(
    temp_location='gs://colab_temp/temp',
    project ='vacio-276411')

table_spec = 'vacio-276411:mainDataset.bulkData'
table_out = 'vacio-276411:mainDataset.outTemp'
table_trips = 'vacio-276411:mainDataset.trips'

table_schema_out = {
    'fields': [{ 'name': 'servicio', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'idVehiculo', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'matricula', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'energia', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
               { 'name': 'latitud', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
               { 'name': 'longitud', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
               { 'name': 'precio', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'tipo', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'categoria', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'imagen', 'type': 'STRING', 'mode': 'NULLABLE'},
               { 'name': 'uoid', 'type': 'STRING', 'mode': 'REQUIRED'},
               { 'name': 'epochTime', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
               { 'name': 'realTime', 'type': 'STRING', 'mode': 'REQUIRED'},
               { 'name': 'geo', 'type': 'GEOGRAPHY', 'mode': 'REQUIRED'},
               { 'name': 'clusterId', 'type': 'INTEGER', 'mode': 'REQUIRED'},
               { 'name': 'clusterLatitude', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
               { 'name': 'clusterLongitude', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
               { 'name': 'timestamp', 'type': 'TIMESTAMP', 'mode': 'REQUIRED'},
               { 'name': 'tipoVehiculo', 'type': 'STRING', 'mode': 'REQUIRED'}
            ]
}

table_schema_trip = {
    'fields': [
              { 'name': 'distancia', 'type': 'FLOAT', 'mode': 'NULLABLE'},
              { 'name': 'servicio', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'idVehiculo', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'matricula', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'energia_start', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'energia_end', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'latitud_start', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'latitud_end', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'longitud_start', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'longitud_end', 'type': 'NUMERIC', 'mode': 'NULLABLE'},
              { 'name': 'precio', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'tipo', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'categoria', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'imagen', 'type': 'STRING', 'mode': 'NULLABLE'},
              { 'name': 'uoid', 'type': 'STRING', 'mode': 'REQUIRED'},
              { 'name': 'epochTime_start', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'epochTime_end', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'realTime_start', 'type': 'STRING', 'mode': 'REQUIRED'},
              { 'name': 'realTime_end', 'type': 'STRING', 'mode': 'REQUIRED'},
              { 'name': 'geo_start', 'type': 'GEOGRAPHY', 'mode': 'REQUIRED'},
              { 'name': 'geo_end', 'type': 'GEOGRAPHY', 'mode': 'REQUIRED'},
              { 'name': 'clusterId_start', 'type': 'INTEGER', 'mode': 'REQUIRED'},
              { 'name': 'clusterId_end', 'type': 'INTEGER', 'mode': 'REQUIRED'},
              { 'name': 'clusterLatitude_start', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'clusterLatitude_end', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'clusterLongitude_start', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'clusterLongitude_end', 'type': 'NUMERIC', 'mode': 'REQUIRED'},
              { 'name': 'timestamp_start', 'type': 'TIMESTAMP', 'mode': 'REQUIRED'},
              { 'name': 'timestamp_end', 'type': 'TIMESTAMP', 'mode': 'REQUIRED'},
              { 'name': 'tipoVehiculo', 'type': 'STRING', 'mode': 'REQUIRED'}
            ]            
}



#query ='SELECT * FROM `vacio-276411.mainDataset.bulkData` WHERE DATE(_PARTITIONTIME) = "2021-04-27" order by idVehiculo,epochTime LIMIT 100'

query = ' '.join(('SELECT * FROM ('
        "SELECT ",
          '*,ROW_NUMBER() OVER(PARTITION BY "a" order by idVehiculo,epochTime)  as tempField',
          'FROM `vacio-276411.mainDataset.bulkData`',
          'WHERE DATE(_PARTITIONTIME) BETWEEN "2021-05-01" AND "2021-05-02"',
      ')WHERE tempField <>1',
      'UNION ALL',
        'SELECT',
          '*,ROW_NUMBER() OVER(PARTITION BY "a" order by idVehiculo,epochTime)+1  as tempField',
          'FROM `vacio-276411.mainDataset.bulkData`',
          'WHERE DATE(_PARTITIONTIME) BETWEEN "2021-05-01" AND "2021-05-02"',
      'ORDER BY tempField,epochTime'))

""" QUERY QUE FUNCIONA EL 2 DE JUNIO """
"""
query = ' '.join(('SELECT * FROM ('
        "SELECT ",
          '*,ROW_NUMBER() OVER(PARTITION BY "a" order by idVehiculo,epochTime)  as tempField',
          'FROM `vacio-276411.mainDataset.bulkData`',
          'WHERE DATE(_PARTITIONTIME) = "2021-05-01"',
      ')WHERE tempField <>1',
      'UNION ALL',
        'SELECT',
          '*,ROW_NUMBER() OVER(PARTITION BY "a" order by idVehiculo,epochTime)+1  as tempField',
          'FROM `vacio-276411.mainDataset.bulkData`',
          'WHERE DATE(_PARTITIONTIME) = "2021-05-01"',
      'ORDER BY tempField,epochTime'))
"""
"""
query = ' '.join(("SELECT * FROM ("
            "SELECT * FROM (",
            "SELECT *,ROW_NUMBER() OVER(PARTITION BY idVehiculo) as TempField",
          "FROM `vacio-276411.mainDataset.bulkData`",
          'WHERE DATE(_PARTITIONTIME) = "2021-04-27"',
          'ORDER BY idVehiculo,timestamp',
        ')',
        'WHERE TempField <>1',
        'UNION ALL',
        'SELECT *,',
        'ROW_NUMBER() OVER(PARTITION BY idVehiculo) as TempField',
        'FROM `vacio-276411.mainDataset.bulkData` WHERE DATE(_PARTITIONTIME) = "2021-04-27"',
        'ORDER BY idVehiculo,timestamp',
        'LIMIT 100'
        ')ORDER BY idVehiculo,timestamp'))
"""

def strip_header_and_newline(text):
  #return ('Hola',text)
  return ((text['idVehiculo']+'*'+str(text['TempField'])), text)

"""
def calcular_distancia(param):

  import math

  R = 6371  # radius of the earth in km
  #print( param[0]['latitud'])
  lat1 = 0
  lat2 = 0
  lon1 = 0
  lon2 = 0
  distancia = 0 #Default value
  if(len(param)==2 and param[0] and param[0]['latitud']):
    lat1 = float (param[0]['latitud'])
  if(len(param)==2 and param[1] and param[1]['latitud']):
    lat2 = float (param[1]['latitud'])
  if(len(param)==2 and param[0] and param[0]['longitud']):
    lon1 = float (param[0]['longitud'])
  if(len(param)==2 and param[1] and param[1]['longitud']):
    lon2 = float (param[1]['longitud'])
  if(lat1 !=0 and lon1 != 0):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    RT=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*RT*math.asin(math.sqrt(a))
  return distancia

def detectar_movimiento(movement):
  return movement['distancia'] > 0.1

def extCalcularDistancia(key,value):
    return {'distancia': calcular_distancia(value) , 'data':value}
"""

def calcular_distancia(key,param):

  #Import inside function
  import math


  R = 6371  # radius of the earth in km
  lat1 = 0
  lat2 = 0
  lon1 = 0
  lon2 = 0
  distancia = 0 #Default value
  if(len(param)==2 and param[0] and param[0]['latitud']):
    lat1 = float (param[0]['latitud'])
  if(len(param)==2 and param[1] and param[1]['latitud']):
    lat2 = float (param[1]['latitud'])
  if(len(param)==2 and param[0] and param[0]['longitud']):
    lon1 = float (param[0]['longitud'])
  if(len(param)==2 and param[1] and param[1]['longitud']):
    lon2 = float (param[1]['longitud'])
  if(lat1 !=0 and lon1 != 0):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    RT=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*RT*math.asin(math.sqrt(a))
    
    return {'distancia': distancia , 'data':param}

def detectar_movimiento(movement):
  #TODO: Hay veces que movement es Nonetype, hay que saber por que
  try:
    return movement['distancia'] > 0.1
  except:
    return False

p2 = beam.Pipeline(options=options)
max_temperatures = (
    p2
    #| 'Create produce' >> beam.Create(['🥕','🥕','🍆','🍅','🍅','🍅'])
    #| 'ReadTable' >> beam.io.ReadFromBigQuery(table=table_spec)
    | 'QueryTable' >> beam.io.ReadFromBigQuery( query=query,use_standard_sql=True)
    # Each row is a dictionary where the keys are the BigQuery columns
    #| beam.io.WriteToBigQuery(
    #  table_out,
    #  schema=table_schema_out,
    #  write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
    #  create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
    #| 'Strip header' >> beam.Map(strip_header_and_newline)
    | beam.Map(lambda elem: (elem['tempField'], elem))
    #| 'Group into batches' >> beam.GroupIntoBatches(2)
    | 'Group counts per produce' >> beam.GroupByKey()
    #| beam.Map(lambda elem: (elem['tempField'], {'distancia':calcular_distancia(elem), 'data':elem}))
    #| beam.MapTuple(lambda key, value: {'distancia': calcular_distancia(value) , 'data':value} )
    #| beam.MapTuple( extCalcularDistancia )
    | beam.MapTuple( calcular_distancia )
    | 'Filter nomove' >> beam.Filter(detectar_movimiento)
    | beam.Map(lambda elem: {
          'distancia':elem['distancia'],
          'servicio' : elem['data'][0]['servicio'],
          'idVehiculo' : elem['data'][0]['idVehiculo'],
          'matricula' : elem['data'][0]['matricula'],
          'energia_start' : elem['data'][0]['energia'],
          'energia_end' : elem['data'][1]['energia'],
          'latitud_start' : elem['data'][0]['latitud'],
          'latitud_end' : elem['data'][1]['latitud'],
          'longitud_start' : elem['data'][0]['longitud'],
          'longitud_end' : elem['data'][1]['longitud'],
          'precio' : elem['data'][0]['precio'],
          'tipo' : elem['data'][0]['tipo'],
          'categoria' : elem['data'][0]['categoria'],
          'imagen' : elem['data'][0]['imagen'],
          'uoid' : elem['data'][0]['uoid'],
          'epochTime_start' : elem['data'][0]['epochTime'],
          'epochTime_end' : elem['data'][0]['epochTime'],
          'realTime_start' : elem['data'][0]['realTime'],
          'realTime_end' : elem['data'][0]['realTime'],
          'geo_start' : elem['data'][0]['geo'],
          'geo_end' : elem['data'][1]['geo'],
          'clusterId_start' : elem['data'][0]['clusterId'],
          'clusterId_end' : elem['data'][1]['clusterId'],
          'clusterLatitude_start' : elem['data'][0]['clusterLatitude'],
          'clusterLatitude_end' : elem['data'][1]['clusterLatitude'],
          'clusterLongitude_start' : elem['data'][0]['clusterLongitude'],
          'clusterLongitude_end' : elem['data'][1]['clusterLongitude'],
          'timestamp_start' : elem['data'][0]['timestamp'],
          'timestamp_end' : elem['data'][1]['timestamp'],
          'tipoVehiculo' : elem['data'][0]['tipoVehiculo']
        } )
    #| beam.io.WriteToText('data/output_mio')
    | beam.io.WriteToBigQuery(
      table_trips,
      schema=table_schema_trip,
      write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
      create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
)




p2.run()

Mounted at /content/drive
